In [1]:
import sys

sys.path.append('../../app/leagues/')

In [2]:
import random
from collections import Counter

from glicko.update import calc_win_prob

from nfl_hensley import build_league

In [3]:
league = build_league()

team_lookup = {
    t.name: t for t in league.teams
}

In [4]:
afc = [
    team_lookup[t] for t in
    ['Chiefs', 'Patriots', 'Texans', 'Ravens', 'Chargers', 'Colts']
]

nfc = [
    team_lookup[t] for t in
    ['Saints', 'Rams', 'Bears', 'Cowboys', 'Seahawks', 'Eagles']
]

In [5]:
def get_winner(team_a, team_b):
    p = calc_win_prob(team_a.rating, team_b.rating)
    return team_a if p > random.random() else team_b

def run_conference(teams):
    # teams is a list of the 6 seeds
    # Gonna do this the Monte Carlo way b/c less code/thinking :)
    div0 = get_winner(teams[2], teams[5])
    div1 = get_winner(teams[3], teams[4])
    
    if div0 == teams[5]:
        lower_div, higher_div = div0, div1
    else:
        lower_div, higher_div = div1, div0
    
    ch0 = get_winner(teams[0], lower_div)
    ch1 = get_winner(teams[1], higher_div)
    return get_winner(ch0, ch1)

In [6]:
def run_playoffs(afc, nfc):
    afc_champ = run_conference(afc)
    nfc_champ = run_conference(nfc)
    return get_winner(afc_champ, nfc_champ)

In [7]:
winners = [
    run_playoffs(afc, nfc)
    for _ in range(int(1e6))
]

In [9]:
for t, count in sorted(Counter(winners).items(), reverse=True, key=lambda t: t[1]):
    print(f'{count / len(winners):.06f}\t{t}')

0.173784	Team(name=Saints, rating=(1622.2506077092312, 5611.737038922921))
0.172482	Team(name=Chiefs, rating=(1617.8262887350143, 5777.087836068869))
0.164673	Team(name=Patriots, rating=(1614.8777467699877, 6422.9258324619095))
0.136643	Team(name=Rams, rating=(1595.1493081532103, 5638.337109199486))
0.055233	Team(name=Ravens, rating=(1579.1415676404824, 5628.188412486042))
0.054203	Team(name=Seahawks, rating=(1569.325322311265, 5692.750401483499))
0.052313	Team(name=Chargers, rating=(1575.6683520717681, 5686.877352955613))
0.051420	Team(name=Eagles, rating=(1573.0877526752258, 5763.175694404063))
0.049454	Team(name=Bears, rating=(1568.2953769418104, 5658.668224549107))
0.035592	Team(name=Texans, rating=(1531.140310186461, 5632.326645701227))
0.030325	Team(name=Cowboys, rating=(1529.6153074263095, 5562.54489659418))
0.023878	Team(name=Colts, rating=(1504.7960540254635, 5722.50979574115))
